# Use Case 1: Classification Pipeline 

In [1]:
pip install groq

   ---------------------------------------- 0.0/129.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/129.4 kB ? eta -:--:--
   ------------ -------------------------- 41.0/129.4 kB 393.8 kB/s eta 0:00:01
   --------------------------- ----------- 92.2/129.4 kB 751.6 kB/s eta 0:00:01
   -------------------------------------- 129.4/129.4 kB 692.6 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install groq pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
from groq import Groq

# Replace with your Groq API key
client = Groq(api_key="Replace with your Groq API key")


In [15]:
import json
import re
from groq import Groq

# Configure Groq API
client = Groq(api_key="Replace with your Groq API key")  # Replace with your actual Groq API key

# Initialize model
model_name = "llama3-8b-8192"

def classify_feedback(feedback: str):
    prompt = f"""
    You are a text classification assistant.
    Classify the following customer feedback as either "Complaint" or "Praise".
    Respond only in this exact JSON format:
    {{
      "label": "Complaint" | "Praise",
      "confidence": float
    }}

    Feedback: "{feedback}"
    """

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )

        response_text = response.choices[0].message.content.strip()
        json_string = re.search(r"\{.*\}", response_text, re.DOTALL).group()
        return json.loads(json_string)

    except Exception as e:
        return {"error": str(e), "raw_output": response_text if 'response_text' in locals() else "No output"}


{'label': 'Praise', 'confidence': 0.9}


In [16]:
# Example usage
feedback_text = "Your customer support was amazing and fixed my issue very quickly!"
result = classify_feedback(feedback_text)
print(result)

{'label': 'Praise', 'confidence': 0.95}


In [17]:
# Example usage
feedback_text = "Not satisfied with the quality of the product"
result = classify_feedback(feedback_text)
print(result)

{'label': 'Complaint', 'confidence': 0.95}


In [22]:
import json
import re
from groq import Groq
from sklearn.metrics import precision_score, recall_score, f1_score

# Initialize Groq client
client = Groq(api_key="Replace with your Groq API key")  # Replace with your actual Groq API key
model_name = "llama3-8b-8192"

def classify_feedback(feedback: str):
    prompt = f"""
    You are a text classification assistant.
    Classify the following customer feedback as either "Complaint" or "Praise".
    Respond only in this exact JSON format:
    {{
      "label": "Complaint" | "Praise",
      "confidence": float between 0.90 and 0.98
    }}

    Feedback: "{feedback}"
    """

    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )

        response_text = response.choices[0].message.content.strip()
        json_string = re.search(r"\{.*\}", response_text, re.DOTALL).group()
        return json.loads(json_string)['label']
    except Exception as e:
        print(f"Error: {e}")
        return "Unknown"

# --- Sample dataset: 30 labeled examples
# Each item is a tuple: (feedback_text, true_label)
dataset = [
    ("The product stopped working after one day.", "Complaint"),
    ("Excellent customer support!", "Praise"),
    ("I'm not happy with the service.", "Complaint"),
    ("Fast delivery and well-packaged.", "Praise"),
    ("Worst experience ever.", "Complaint"),
    ("They were very polite and helpful.", "Praise"),
    ("My issue was not resolved even after multiple attempts.", "Complaint"),
    ("Great quality and affordable price.", "Praise"),
    ("Rude staff at the service center.", "Complaint"),
    ("Loved the quick response time.", "Praise"),
    ("No one answered my call.", "Complaint"),
    ("Thank you for resolving my issue so quickly!", "Praise"),
    ("The item was defective and returned.", "Complaint"),
    ("Very satisfied with the product.", "Praise"),
    ("They didn’t listen to my concerns.", "Complaint"),
    ("Great shopping experience!", "Praise"),
    ("Repeated issues with billing.", "Complaint"),
    ("Smooth checkout process.", "Praise"),
    ("The packaging was terrible.", "Complaint"),
    ("The representative went above and beyond.", "Praise"),
    ("Misleading advertisement.", "Complaint"),
    ("Prompt and friendly service.", "Praise"),
    ("The replacement never arrived.", "Complaint"),
    ("Absolutely love this product!", "Praise"),
    ("Broken when it arrived.", "Complaint"),
    ("The staff were knowledgeable and helpful.", "Praise"),
    ("It took weeks to get a reply.", "Complaint"),
    ("Customer service was fantastic!", "Praise"),
    ("They ignored my refund request.", "Complaint"),
    ("Very happy with my purchase.", "Praise")
]

# --- Classification loop
true_labels = []
predicted_labels = []

for feedback, true_label in dataset:
    predicted_label = classify_feedback(feedback)
    true_labels.append(true_label)
    predicted_labels.append(predicted_label)
    print(f"Feedback: {feedback}\nPredicted: {predicted_label}, Actual: {true_label}\n")

# --- Scoring
precision = precision_score(true_labels, predicted_labels, pos_label="Praise", average="binary")
recall = recall_score(true_labels, predicted_labels, pos_label="Praise", average="binary")
f1 = f1_score(true_labels, predicted_labels, pos_label="Praise", average="binary")

print(f"\nPrecision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Feedback: The product stopped working after one day.
Predicted: Complaint, Actual: Complaint

Feedback: Excellent customer support!
Predicted: Praise, Actual: Praise

Feedback: I'm not happy with the service.
Predicted: Complaint, Actual: Complaint

Feedback: Fast delivery and well-packaged.
Predicted: Praise, Actual: Praise

Feedback: Worst experience ever.
Predicted: Complaint, Actual: Complaint

Feedback: They were very polite and helpful.
Predicted: Praise, Actual: Praise

Feedback: My issue was not resolved even after multiple attempts.
Predicted: Complaint, Actual: Complaint

Feedback: Great quality and affordable price.
Predicted: Praise, Actual: Praise

Feedback: Rude staff at the service center.
Predicted: Complaint, Actual: Complaint

Feedback: Loved the quick response time.
Predicted: Praise, Actual: Praise

Feedback: No one answered my call.
Predicted: Complaint, Actual: Complaint

Feedback: Thank you for resolving my issue so quickly!
Predicted: Praise, Actual: Praise

Fee

In [25]:
from groq import Groq

# Initialize Groq client
client = Groq(api_key="Replace with your Groq API key")  # Replace with your actual Groq API key

# Define the prompt
prompt = """
You are a customer feedback generator.

Your task is to generate realistic, natural-sounding customer complaints. These should reflect real frustrations, negative experiences, or product/service issues people might encounter.

Guidelines:
- Each example should be one to two sentences long.
- Do not use the word “Complaint” in the text.
- Do not include labels or numbering — only the raw feedback text.
- Vary the topics: shipping issues, broken items, poor service, delays, billing problems, etc.

Generate 100 unique customer complaints.
"""

# Use chat-based generation
response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=3072,
)

# Extract the generated text
generated_text = response.choices[0].message.content.strip()

# Remove the heading line if present
lines = generated_text.splitlines()
if lines[0].lower().startswith("here are"):
    lines = lines[1:]

# Clean and filter complaints
complaints = [line.strip("-•*0123456789. ").strip() for line in lines if line.strip()]

# Show first 5 complaints
for i, complaint in enumerate(complaints[:5], 1):
    print(f"{i}. {complaint}")


1. I was really disappointed when I received my order and the item was damaged during shipping
2. The new smartphone I purchased from your company has a faulty screen that keeps freezing and needs to be replaced
3. The customer service representative I spoke with was extremely rude and unhelpful when I tried to return a defective product
4. I was charged for a subscription service that I never signed up for, and I have no idea how it happened
5. The food I received through your meal kit service was spoiled and inedible due to poor packaging


In [26]:
prompt = f"""
You are a text classification assistant.
Classify the following customer feedback as either "Complaint" or "Praise".

Guidelines:
- If the feedback includes both positive and negative elements, classify it based on the dominant sentiment.
- If the customer expresses dissatisfaction, frustration, delay, defect, or unresolved issue — classify it as a "Complaint".
- Only classify as "Praise" if the feedback is clearly positive and free of issues.

Respond only in this exact JSON format:
{{
  "label": "Complaint" | "Praise",
  "confidence": float between 0.90 and 0.98
}}

Feedback: "{feedback}"
"""


In [27]:
# After model prediction
if any(keyword in feedback.lower() for keyword in ["not", "issue", "problem", "delay", "unresolved", "defective"]):
    predicted_label = "Complaint"

In [33]:
import json
import random
from groq import Groq

# Initialize Groq client
client = Groq(api_key="Replace with your Groq API key")  # Replace with your actual Groq API key

# Define the prompt
prompt = """
You are a customer feedback generator.

Your task is to generate realistic, natural-sounding customer complaints. These should reflect real frustrations, negative experiences, or product/service issues people might encounter.

Guidelines:
- Each example should be one to two sentences long.
- Do not use the word “Complaint” in the text.
- Do not include labels or numbering — only the raw feedback text.
- Vary the topics: shipping issues, broken items, poor service, delays, billing problems, etc.

Generate 100 unique customer complaints.
"""

# Use chat-based generation
response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=3072,
)

# Extract the generated text
generated_text = response.choices[0].message.content.strip()

# Remove the heading line if present
lines = generated_text.splitlines()
if lines and lines[0].lower().startswith("here are"):
    lines = lines[1:]

# Clean and filter complaints
complaints = [line.strip("-•*0123456789. ").strip() for line in lines if line.strip()]

# Save complaints to a JSON file
with open("complaints_100.json", "w", encoding="utf-8") as f:
    json.dump(complaints, f, ensure_ascii=False, indent=2)

# Randomly select 5 complaints to display
sample_complaints = random.sample(complaints, min(5, len(complaints)))

print("\nRandomly selected complaints:")
for i, complaint in enumerate(sample_complaints, 1):
    print(f"{i}. {complaint}")



Randomly selected complaints:
1. I'm still waiting to receive my refund, despite being promised it would be processed within
2. The company's return policy is completely unreasonable and makes it impossible to return an item that's defective
3. The product is overpriced for the quality you receive, and I feel ripped off
4. The company's return policy is completely unreasonable and makes it impossible to return an item that's defective
5. I've tried to use the product multiple times, but it keeps malfunctioning and won't work properly


In [34]:
import json
import random
from groq import Groq

# Initialize Groq client
client = Groq(api_key="Replace with your Groq API key")  # Replace with your actual Groq API key

# Define the prompt
prompt = """
You are a customer feedback generator.

Your task is to generate realistic, natural-sounding customer complaints. These should reflect real frustrations, negative experiences, or product/service issues people might encounter.

Guidelines:
- Each example should be one to two sentences long.
- Do not use the word “Complaint” in the text.
- Do not include labels or numbering — only the raw feedback text.
- Vary the topics: shipping issues, broken items, poor service, delays, billing problems, etc.

Generate 100 unique customer complaints.
"""

# Use chat-based generation
response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ],
    temperature=0.7,
    max_tokens=3072,
)

# Extract the generated text
generated_text = response.choices[0].message.content.strip()

# Remove the heading line if present
lines = generated_text.splitlines()
if lines and lines[0].lower().startswith("here are"):
    lines = lines[1:]

# Clean and filter complaints
complaints = [line.strip("-•*0123456789. ").strip() for line in lines if line.strip()]

# Save complaints to a JSON file
with open("complaints_100.json", "w", encoding="utf-8") as f:
    json.dump(complaints, f, ensure_ascii=False, indent=2)

# Randomly select 5 complaints to display
sample_complaints = random.sample(complaints, min(5, len(complaints)))

print("\nRandomly selected complaints:")
for i, complaint in enumerate(sample_complaints, 1):
    print(f"{i}. {complaint}")

# Select 3 random complaints as misclassified cases
misclassified_cases = random.sample(complaints, min(3, len(complaints)))

# Analyze root causes and propose fixes for misclassified cases
# (Note: These are example root causes and fixes for demonstration purposes)
analysis = [
    {
        "feedback": misclassified_cases[0],
        "root_cause": "Prompt ambiguity — complaint tone can be mistaken for neutral or positive feedback.",
        "fix": "Clarify in prompt that complaints must explicitly state dissatisfaction or issue."
    },
    {
        "feedback": misclassified_cases[1],
        "root_cause": "Class imbalance — too few examples of billing or app issues causing misclassification.",
        "fix": "Add more varied examples in the prompt to balance complaint categories."
    },
    {
        "feedback": misclassified_cases[2],
        "root_cause": "Complex sentences with mixed sentiments — praise mixed with complaint confuses the classifier.",
        "fix": "Post-processing rule: classify as complaint if any unresolved negative issue is mentioned."
    },
]

print("\n--- Misclassified Case Analysis ---")
for i, case in enumerate(analysis, 1):
    print(f"\nCase {i}:")
    print("Feedback:", case["feedback"])
    print("Root Cause:", case["root_cause"])
    print("Proposed Fix:", case["fix"])


Randomly selected complaints:
1. I've tried to contact your support team multiple times, but no one has gotten back to me and I'm starting to feel like I'm being ignored
2. I was surprised to find that the item I received was not what I ordered, and I'm still waiting to hear back from your team
3. I had a terrible experience with the product, it stopped working after only a few uses
4. I'm still waiting to hear back from your team about the issue I'm having with my order, and it's already been over a week
5. I'm extremely frustrated with the slow shipping and lack of communication from your team - I've been waiting for over two weeks for my order to arrive

--- Misclassified Case Analysis ---

Case 1:
Feedback: I'm extremely frustrated with the slow shipping and lack of communication from your team - I've been waiting for over two weeks for my order to arrive
Root Cause: Prompt ambiguity — complaint tone can be mistaken for neutral or positive feedback.
Proposed Fix: Clarify in prompt

In [35]:
import json
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Load complaints from file
with open("complaints_100.json", "r", encoding="utf-8") as f:
    complaints = json.load(f)

# Step 1: Heuristic labeling function
def heuristic_label(complaint):
    complaint_lower = complaint.lower()
    if any(word in complaint_lower for word in ["late", "delay", "slow", "lost", "shipping", "delivery"]):
        return "A"  # Prompt ambiguity (often mixed or unclear tone)
    elif any(word in complaint_lower for word in ["bill", "refund", "charge", "overcharged", "billing"]):
        return "B"  # Class imbalance (billing related)
    elif any(word in complaint_lower for word in ["good", "thank", "helpful", "nice"]):
        return "C"  # Mixed sentiment (praise + complaint)
    else:
        # Default to A for unknown/ambiguous
        return "A"

# Step 2: Create labeled dataset
training_data = [{"text": c, "label": heuristic_label(c)} for c in complaints]

# Extract features and labels
texts = [item["text"] for item in training_data]
labels = [item["label"] for item in training_data]

# Vectorize
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

# Train classifier
clf = LogisticRegression(max_iter=200)
clf.fit(X, labels)

# Fix mapping
fixes = {
    "A": "Clarify in prompt that complaints must explicitly state dissatisfaction or issue.",
    "B": "Add more varied examples in the prompt to balance complaint categories.",
    "C": "Post-processing rule: classify as complaint if any unresolved negative issue is mentioned."
}

# Select 3 random complaints as misclassified cases for demo
misclassified_cases = random.sample(complaints, min(3, len(complaints)))

# Analyze misclassified cases automatically
print("\n--- Misclassified Case Analysis (automated) ---")
for i, feedback in enumerate(misclassified_cases, 1):
    X_test = vectorizer.transform([feedback])
    pred_label = clf.predict(X_test)[0]
    proposed_fix = fixes[pred_label]
    print(f"\nCase {i}:")
    print("Feedback:", feedback)
    print("Predicted Root Cause:", pred_label)
    print("Proposed Fix:", proposed_fix)



--- Misclassified Case Analysis (automated) ---

Case 1:
Feedback: I was surprised to find that the item I received was not what I ordered, and I'm still waiting to hear back from your team
Predicted Root Cause: A
Proposed Fix: Clarify in prompt that complaints must explicitly state dissatisfaction or issue.

Case 2:
Feedback: I was expecting a higher level of quality from a product that is supposed to be a premium item
Predicted Root Cause: A
Proposed Fix: Clarify in prompt that complaints must explicitly state dissatisfaction or issue.

Case 3:
Feedback: I was shocked to find that my order was delayed by over a week, despite the estimated delivery date being promised to me
Predicted Root Cause: A
Proposed Fix: Clarify in prompt that complaints must explicitly state dissatisfaction or issue.


In [36]:
import json
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Load complaints from JSON file
with open("complaints_100.json", "r", encoding="utf-8") as f:
    complaints = json.load(f)

# Heuristic function to assign root cause labels
def heuristic_label(complaint):
    complaint_lower = complaint.lower()
    if any(word in complaint_lower for word in ["late", "delay", "slow", "lost", "shipping", "delivery"]):
        return "A"  # Prompt ambiguity
    elif any(word in complaint_lower for word in ["bill", "refund", "charge", "overcharged", "billing"]):
        return "B"  # Class imbalance
    elif any(word in complaint_lower for word in ["good", "thank", "helpful", "nice"]):
        return "C"  # Mixed sentiment
    else:
        return "A"  # Default to A if uncertain

# Create training data with heuristic labels
training_data = [{"text": c, "label": heuristic_label(c)} for c in complaints]
texts = [item["text"] for item in training_data]
labels = [item["label"] for item in training_data]

# Vectorize the complaints using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)

# Train Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X, labels)

# Mapping predicted labels to fixes
fixes = {
    "A": "Clarify in prompt that complaints must explicitly state dissatisfaction or issue.",
    "B": "Add more varied examples in the prompt to balance complaint categories.",
    "C": "Post-processing rule: classify as complaint if any unresolved negative issue is mentioned."
}

# Select 3 random complaints to simulate misclassification analysis
misclassified_cases = random.sample(complaints, 3)

# Analyze with the trained model
print("\n--- Misclassified Case Analysis (Naive Bayes) ---")
for i, feedback in enumerate(misclassified_cases, 1):
    X_test = vectorizer.transform([feedback])
    pred_label = clf.predict(X_test)[0]
    proposed_fix = fixes[pred_label]
    print(f"\nCase {i}:")
    print("Feedback:", feedback)
    print("Predicted Root Cause:", pred_label)
    print("Proposed Fix:", proposed_fix)


--- Misclassified Case Analysis (Naive Bayes) ---

Case 1:
Feedback: I was surprised to find that the item I received was not what I ordered, and I'm still waiting to hear back from your team
Predicted Root Cause: A
Proposed Fix: Clarify in prompt that complaints must explicitly state dissatisfaction or issue.

Case 2:
Feedback: I had a terrible experience with the product, it stopped working after only a few uses
Predicted Root Cause: A
Proposed Fix: Clarify in prompt that complaints must explicitly state dissatisfaction or issue.

Case 3:
Feedback: I'm still waiting to hear back from your team about the issue I'm having with my order, and it's already been over a week
Predicted Root Cause: A
Proposed Fix: Clarify in prompt that complaints must explicitly state dissatisfaction or issue.


In [37]:
import json
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Load complaints from file
with open("complaints_100.json", "r", encoding="utf-8") as f:
    complaints = json.load(f)

# Heuristic labeling function
def heuristic_label(text):
    text = text.lower()
    if any(word in text for word in ["delay", "late", "lost", "shipping", "delivery"]):
        return "A"  # Prompt ambiguity
    elif any(word in text for word in ["billing", "refund", "overcharged", "charge"]):
        return "B"  # Class imbalance
    elif any(word in text for word in ["good", "thank", "nice", "helpful"]):
        return "C"  # Mixed sentiment
    else:
        return "A"  # Default category

# Create dataset
data = [{"text": complaint, "label": heuristic_label(complaint)} for complaint in complaints]
texts = [d["text"] for d in data]
labels = [d["label"] for d in data]

# Vectorize text
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)
y = labels

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
logreg = LogisticRegression(max_iter=1000)
nb = MultinomialNB()

# Train models
logreg.fit(X_train, y_train)
nb.fit(X_train, y_train)

# Predict
logreg_preds = logreg.predict(X_test)
nb_preds = nb.predict(X_test)

# Evaluate
logreg_acc = accuracy_score(y_test, logreg_preds)
nb_acc = accuracy_score(y_test, nb_preds)

# Results
print("\n Model Comparison Results")
print("--------------------------")
print(f"Logistic Regression Accuracy: {logreg_acc:.2f}")
print(f"Naive Bayes Accuracy:         {nb_acc:.2f}")

print("\n Classification Report (Logistic Regression):")
print(classification_report(y_test, logreg_preds))

print("\n Classification Report (Naive Bayes):")
print(classification_report(y_test, nb_preds))

# Final verdict
better_model = "Logistic Regression" if logreg_acc > nb_acc else "Naive Bayes"
print(f"\n More Accurate Model: {better_model}")



 Model Comparison Results
--------------------------
Logistic Regression Accuracy: 1.00
Naive Bayes Accuracy:         1.00

 Classification Report (Logistic Regression):
              precision    recall  f1-score   support

           A       1.00      1.00      1.00        16
           C       1.00      1.00      1.00         1

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17


 Classification Report (Naive Bayes):
              precision    recall  f1-score   support

           A       1.00      1.00      1.00        16
           C       1.00      1.00      1.00         1

    accuracy                           1.00        17
   macro avg       1.00      1.00      1.00        17
weighted avg       1.00      1.00      1.00        17


 More Accurate Model: Naive Bayes


# Use Case 2: LLM Fine-Tuning Pipeline

In [1]:
!pip install faker

In [2]:
import json

# Define properly aligned ticket-action pairs
ticket_action_pairs = [
    {
        "ticket": "I'm unable to connect to the company VPN from home since this morning.",
        "actions": [
            "- Check the user's network settings and VPN credentials",
            "- Restart the VPN service and verify server availability",
            "- Escalate to network admin if connection still fails"
        ]
    },
    {
        "ticket": "The system crashes every time I try to upload a file larger than 10MB.",
        "actions": [
            "- Reproduce the error with sample files",
            "- Check for application logs related to file handling",
            "- Patch the uploader module or escalate to dev team"
        ]
    },
    {
        "ticket": "Emails are not syncing on my mobile device even after reinstalling the app.",
        "actions": [
            "- Confirm email account configuration on the device",
            "- Clear app cache and verify sync settings",
            "- Contact IT to check for server-side sync issues"
        ]
    },
    {
        "ticket": "The shared network drive is inaccessible; it's saying permission denied.",
        "actions": [
            "- Verify user permissions on the shared drive",
            "- Test drive access from another user account",
            "- Escalate to IT to reset access controls"
        ]
    },
    {
        "ticket": "My password reset link expired before I could use it.",
        "actions": [
            "- Re-send password reset link",
            "- Check for time zone discrepancies affecting expiration",
            "- Log a request for manual reset by support team"
        ]
    }
]

# Expand to 50 synthetic examples by repeating and varying the above
tickets_data = []
for i in range(50):
    pair = ticket_action_pairs[i % len(ticket_action_pairs)]  # Cycle through defined pairs
    record = {
        "messages": [
            {"role": "user", "content": pair["ticket"]},
            {"role": "assistant", "content": "\n".join(pair["actions"])}
        ]
    }
    tickets_data.append(record)

# Save to JSONL
with open("support_tickets_50.jsonl", "w", encoding="utf-8") as f:
    for record in tickets_data:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

# Display 5 sample records
print("Sample records:\n")
for example in tickets_data[:5]:
    print(json.dumps(example, indent=2))


Sample records:

{
  "messages": [
    {
      "role": "user",
      "content": "I'm unable to connect to the company VPN from home since this morning."
    },
    {
      "role": "assistant",
      "content": "- Check the user's network settings and VPN credentials\n- Restart the VPN service and verify server availability\n- Escalate to network admin if connection still fails"
    }
  ]
}
{
  "messages": [
    {
      "role": "user",
      "content": "The system crashes every time I try to upload a file larger than 10MB."
    },
    {
      "role": "assistant",
      "content": "- Reproduce the error with sample files\n- Check for application logs related to file handling\n- Patch the uploader module or escalate to dev team"
    }
  ]
}
{
  "messages": [
    {
      "role": "user",
      "content": "Emails are not syncing on my mobile device even after reinstalling the app."
    },
    {
      "role": "assistant",
      "content": "- Confirm email account configuration on the device\n

In [3]:
!pip install transformers accelerate datasets

In [4]:
# Hyperparameter Configuration for Fine-tuning LLaMA 2
hyperparameters = {
    "model_name_or_path": "meta-llama/Llama-2-7b-chat-hf",  # Choose either 7B or 13B model
    "learning_rate": 3e-5,
    "epochs": 4,
    "batch_size": 4,
    "max_seq_length": 512,
    "optimizer": "AdamW",
    "gradient_accumulation_steps": 8,
    "evaluation_strategy": "steps",
    "save_steps": 100,
    "logging_steps": 10,
    "eval_steps": 100,
    "output_dir": "./llama2-support-ticket-summarizer",
    "train_file": "support_tickets_50_formatted.jsonl",  # Path to your training data
    "validation_file": "support_tickets_val.jsonl",    # Path to your validation data
}

# Print the hyperparameters for reference
import json
print(json.dumps(hyperparameters, indent=2))

# Build the CLI command based on the above parameters
cli_command = f"""
accelerate launch run_clm.py \\
  --model_name_or_path {hyperparameters['model_name_or_path']} \\
  --train_file {hyperparameters['train_file']} \\
  --validation_file {hyperparameters['validation_file']} \\
  --do_train \\
  --do_eval \\
  --num_train_epochs {hyperparameters['epochs']} \\
  --per_device_train_batch_size {hyperparameters['batch_size']} \\
  --per_device_eval_batch_size {hyperparameters['batch_size']} \\
  --learning_rate {hyperparameters['learning_rate']} \\
  --max_seq_length {hyperparameters['max_seq_length']} \\
  --output_dir {hyperparameters['output_dir']} \\
  --logging_steps {hyperparameters['logging_steps']} \\
  --save_steps {hyperparameters['save_steps']} \\
  --evaluation_strategy {hyperparameters['evaluation_strategy']} \\
  --eval_steps {hyperparameters['eval_steps']} \\
  --overwrite_output_dir
"""

# Print the CLI command for running the fine-tuning job
print("\nCLI Command to run fine-tuning:")
print(cli_command)

{
  "model_name_or_path": "meta-llama/Llama-2-7b-chat-hf",
  "learning_rate": 3e-05,
  "epochs": 4,
  "batch_size": 4,
  "max_seq_length": 512,
  "optimizer": "AdamW",
  "gradient_accumulation_steps": 8,
  "evaluation_strategy": "steps",
  "save_steps": 100,
  "logging_steps": 10,
  "eval_steps": 100,
  "output_dir": "./llama2-support-ticket-summarizer",
  "train_file": "support_tickets_50_formatted.jsonl",
  "validation_file": "support_tickets_val.jsonl"
}

CLI Command to run fine-tuning:

accelerate launch run_clm.py \
  --model_name_or_path meta-llama/Llama-2-7b-chat-hf \
  --train_file support_tickets_50_formatted.jsonl \
  --validation_file support_tickets_val.jsonl \
  --do_train \
  --do_eval \
  --num_train_epochs 4 \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4 \
  --learning_rate 3e-05 \
  --max_seq_length 512 \
  --output_dir ./llama2-support-ticket-summarizer \
  --logging_steps 10 \
  --save_steps 100 \
  --evaluation_strategy steps \
  --eval_steps

In [5]:
import json
import random

# Load your original dataset
input_file = 'support_tickets_50.jsonl'

with open(input_file, 'r', encoding='utf-8') as f:
    tickets_data = [json.loads(line) for line in f]

# Split the data into 80% train and 20% validation
random.shuffle(tickets_data)

train_size = int(0.8 * len(tickets_data))
train_data = tickets_data[:train_size]
val_data = tickets_data[train_size:]

# Save the training and validation data to separate files
with open('support_tickets_train.jsonl', 'w', encoding='utf-8') as f:
    for record in train_data:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

with open('support_tickets_val.jsonl', 'w', encoding='utf-8') as f:
    for record in val_data:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print("Data has been split into training and validation sets.")

Data has been split into training and validation sets.


In [8]:
!pip install sentencepiece

In [9]:
import sentencepiece
print(sentencepiece.__version__)

0.2.0


In [10]:
pip install sentencepiece --upgrade

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.


In [19]:
!pip install tensorflow
!pip install numpy
!pip install scikit-learn

   ---------------------------------------- 0.0/413.4 kB ? eta -:--:--
    --------------------------------------- 10.2/413.4 kB ? eta -:--:--
   --- ----------------------------------- 41.0/413.4 kB 653.6 kB/s eta 0:00:01
   ---------------- ----------------------- 174.1/413.4 kB 1.7 MB/s eta 0:00:01
   ----------------------------------- ---- 368.6/413.4 kB 2.9 MB/s eta 0:00:01
   ---------------------------------------- 413.4/413.4 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.4
    Uninstalling protobuf-5.29.4:
      Successfully uninstalled protobuf-5.29.4


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.7 which is incompatible.


In [30]:
# Extract both user queries and assistant responses
user_queries = []
assistant_responses = []

for index, row in df.iterrows():
    messages = row['messages']
    user_message = None
    assistant_message = None
    
    for message in messages:
        if message['role'] == 'user':
            user_message = message['content']
        elif message['role'] == 'assistant':
            assistant_message = message['content']
        
        # Only append when both parts are found
        if user_message and assistant_message:
            user_queries.append(user_message)
            assistant_responses.append(assistant_message)
            break

# Create a DataFrame for both inputs and responses
training_data = pd.DataFrame({
    'user_query': user_queries,
    'assistant_response': assistant_responses
})

# Check the new DataFrame
print(training_data.head())

                                          user_query  \
0  I'm unable to connect to the company VPN from ...   
1  The system crashes every time I try to upload ...   
2  Emails are not syncing on my mobile device eve...   
3  The shared network drive is inaccessible; it's...   
4  My password reset link expired before I could ...   

                                  assistant_response  
0  - Check the user's network settings and VPN cr...  
1  - Reproduce the error with sample files\n- Che...  
2  - Confirm email account configuration on the d...  
3  - Verify user permissions on the shared drive\...  
4  - Re-send password reset link\n- Check for tim...  


In [31]:
# Save the dataset in the required JSONL format for OpenAI API
training_data.to_json('training_data.jsonl', orient='records', lines=True)

In [33]:
import pandas as pd
import json

# Load your dataset
df = pd.read_json('support_tickets_50.jsonl', lines=True)

# Extract 'user' messages
texts = []
for index, row in df.iterrows():
    messages = row['messages']
    for message in messages:
        if message['role'] == 'user':
            texts.append(message['content'])

# Convert the list of user messages into a DataFrame
user_queries_df = pd.DataFrame(texts, columns=['ticket_text'])

# Prepare data for fine-tuning (we assume assistant responses can be "mocked" for this example)
fine_tuning_data = []

# Mock assistant responses (this can be customized or extracted from a separate column if available)
for text in user_queries_df['ticket_text']:
    prompt = f"User query: {text}\n\nAssistant response:"
    completion = "Please check your network settings and try again."  # Example response (can be tailored)
    fine_tuning_data.append({"prompt": prompt, "completion": completion})

# Save data as a JSONL file for fine-tuning
with open("fine_tuning_data.jsonl", "w") as f:
    for entry in fine_tuning_data:
        json.dump(entry, f)
        f.write("\n")

print("Data saved in fine_tuning_data.jsonl for fine-tuning.")

Data saved in fine_tuning_data.jsonl for fine-tuning.


In [52]:
import json
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Load the dataset (assuming it is in 'data.jsonl' file)
file_path = 'support_tickets_50.jsonl' 
data = []

with open(file_path, 'r') as f:
    for line in f:
        data.append(json.loads(line))

# Extracting 'content' from both user and assistant
input_data = [entry['messages'][0]['content'] for entry in data]  # User's message
output_data = [entry['messages'][1]['content'] for entry in data]  # Assistant's response

# Tokenization for input data (user messages)
input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_data)
input_sequences = input_tokenizer.texts_to_sequences(input_data)

# Tokenization for output data (assistant responses)
output_tokenizer = Tokenizer()
output_tokenizer.fit_on_texts(output_data)
output_sequences = output_tokenizer.texts_to_sequences(output_data)

# Define vocab sizes
vocab_size_input = len(input_tokenizer.word_index) + 1
vocab_size_output = len(output_tokenizer.word_index) + 1

# Maximum sequence lengths
max_input_length = max([len(seq) for seq in input_sequences])
max_output_length = max([len(seq) for seq in output_sequences])

# Pad the sequences to make sure they're all the same length
input_data_padded = pad_sequences(input_sequences, maxlen=max_input_length)

# **Pad target sequences to max_input_length or any specific length you'd like**
output_data_padded = pad_sequences(output_sequences, maxlen=max_input_length)

# One-hot encode the output data
output_data_one_hot = to_categorical(output_data_padded, num_classes=vocab_size_output)

# Build the model
model = Sequential()

# Encoder: Embedding layer for input (user messages)
model.add(Embedding(input_dim=vocab_size_input, output_dim=128, input_length=max_input_length))

# LSTM layer for encoding the input sequence
model.add(LSTM(128, return_sequences=True))  # return_sequences=True for sequence output
model.add(LSTM(128, return_sequences=True))  # another LSTM layer for encoding

# Decoder: Dense layer for predicting the next word in the sequence (assistant response)
model.add(Dense(vocab_size_output, activation='softmax'))  # Output for each time step

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(input_data_padded, output_data_one_hot, epochs=10, batch_size=32)


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.0319 - loss: 4.3422
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0862 - loss: 4.3271
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0854 - loss: 4.3042
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0854 - loss: 4.2623
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.0869 - loss: 4.1785
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.0891 - loss: 4.0484
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.0824 - loss: 3.9489
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0817 - loss: 3.9001
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.0824 - loss: 3.8409
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.1001 - loss: 3.7622


In [53]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

# Model Architecture Improvements
model = Sequential()

# Embedding Layer
model.add(Embedding(input_dim=vocab_size_input, output_dim=128, input_length=max_input_length))

# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(128, return_sequences=True)))  # Bidirectional LSTM
model.add(Dropout(0.3))  # Dropout for regularization

# LSTM Layer
model.add(LSTM(128, return_sequences=True))  # Another LSTM layer
model.add(Dropout(0.3))  # Dropout for regularization

# Output Layer
model.add(Dense(vocab_size_output, activation='softmax'))  # Output layer for each time step

# Compile the model with a lower learning rate
optimizer = Adam(learning_rate=0.001)  # You can try tuning the learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(input_data_padded, output_data_one_hot, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 680ms/step - accuracy: 0.0173 - loss: 4.3431 - val_accuracy: 0.1429 - val_loss: 4.3279
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.0920 - loss: 4.3271 - val_accuracy: 0.1000 - val_loss: 4.3080
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.1025 - loss: 4.3066 - val_accuracy: 0.1000 - val_loss: 4.2760
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.0990 - loss: 4.2728 - val_accuracy: 0.1000 - val_loss: 4.2179
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.0908 - loss: 4.2121 - val_accuracy: 0.1000 - val_loss: 4.1093
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.0847 - loss: 4.1054 - val_accuracy: 0.1000 - val_loss: 3.9569
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.0857 - loss: 3.9514 - val_accuracy: 0.0857 - val_loss: 3.8793
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.0769 - loss: 3.8778 - val_accuracy: 0.0857 - val_loss: 3.7953

In [56]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Load the model (if you have saved it, otherwise continue using your trained model)
# model = load_model('your_model_path')

# Set the temperature and other parameters
temperature = 0.7  # Adjust the temperature for more control over output randomness

def predict_response(input_text, model, max_input_length, input_tokenizer, output_tokenizer):
    # Convert input text to sequences
    input_sequence = input_tokenizer.texts_to_sequences([input_text])
    input_data_padded = pad_sequences(input_sequence, maxlen=max_input_length)
    
    # Get predictions
    predictions = model.predict(input_data_padded, verbose=0)
    
    # Apply temperature to the predictions to reduce repetition
    predictions = predictions / temperature
    predictions = np.exp(predictions)  # Apply softmax temperature scaling
    predictions = predictions / np.sum(predictions, axis=-1, keepdims=True)  # Normalize again

    # Get the word with the highest probability (Argmax for prediction)
    predicted_word_indices = predictions.argmax(axis=-1)

    # Convert indices to words using output_tokenizer
    predicted_words = output_tokenizer.sequences_to_texts(predicted_word_indices)
    
    # Post-process: Format into bullet points
    # Assuming each line contains separate troubleshooting steps
    formatted_output = format_response(predicted_words[0])

    return formatted_output

def format_response(response_text):
    # Post-process to format output as bullet points, removing unnecessary words
    formatted_response = response_text.strip().split()  # Split words into a list

    # Remove redundancy, if any (optional: this could be more complex for better handling)
    # Convert list to set and back to list to remove duplicates, preserving word order.
    unique_words = list(dict.fromkeys(formatted_response))  # Removes duplicates while maintaining order

    # Join into a single string with bullet points
    return "\n- ".join(unique_words)

# Example of usage:
test_data = "The shared network drive is inaccessible; it's saying permission denied."
formatted_response = predict_response(test_data, model, max_input_length, input_tokenizer, output_tokenizer)

print("Formatted Predicted Response:\n", "- " + formatted_response)


Formatted Predicted Response:
 - test
- drive
- access
- from
- another
- user
- account
- escalate
- to
- reset
- controls


In [59]:
!pip install rouge-score

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24971 sha256=9b92ede810921effcd7bffccb2d7bf0d70d5ef29a903f3d99a81624f1a839d03
  Stored in directory: c:\users\manasi\appdata\local\pip\cache\wheels\1e\19\43\8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [60]:
from rouge_score import rouge_scorer

# Assuming you have the validation set and the model's responses
reference_summaries = [
    "Expected assistant summaries for validation data."
]
predicted_summaries = [
    "Generated assistant summaries from the fine-tuned model."
]

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1'], use_stemmer=True)

# Calculate ROUGE-1 F1
scores = [scorer.score(ref, pred) for ref, pred in zip(reference_summaries, predicted_summaries)]
rouge_1_f1 = [score['rouge1'].fmeasure for score in scores]

print(f"ROUGE-1 F1: {sum(rouge_1_f1) / len(rouge_1_f1)}")


ROUGE-1 F1: 0.28571428571428575


In [61]:
# Import necessary libraries
import random

# Sample data for analysis (emulating assistant outputs)
failure_case_1 = {
    "user_query": "I can't connect to the company VPN from home, what should I do?",
    "assistant_response": "Please check your network settings and try again."
}

failure_case_2 = {
    "user_query": "What is the status of my IT support ticket?",
    "assistant_response": "Your support ticket is currently being reviewed by our IT team. They are analyzing the issue to identify the cause, and once they have completed their investigation, they will inform you of the next steps. The expected timeline for resolution is approximately 24 hours, depending on the complexity of the issue. In the meantime, please feel free to check back or contact our team if you have any further questions."
}

# --- Failure Case 1: Missing Action Items ---
# We need to ensure that the assistant provides a full troubleshooting guide.
def analyze_missing_action_items(failure_case):
    print(f"\nUser Query: {failure_case['user_query']}")
    print(f"Assistant Response (Before): {failure_case['assistant_response']}")
    
    # --- Prompt Template Tweak ---
    # Adding more explicit instructions for the model to include a complete troubleshooting guide.
    prompt_tweak = """
    User query: "I can't connect to the company VPN from home, what should I do?"
    Assistant response: "Provide a step-by-step troubleshooting guide with clear actions for the user, including things like checking network settings, verifying VPN credentials, restarting the VPN service, and escalation to IT if necessary."
    """
    
    print("\nProposed Prompt Template Tweak (To guide assistant to provide complete action items):")
    print(prompt_tweak)

    # --- Post-Processing Rule ---
    # Implementing a post-processing rule to enforce the inclusion of key action items.
    required_actions = ["check network", "restart", "contact support", "verify credentials"]

    def enforce_action_items(response):
        for action in required_actions:
            if action not in response.lower():
                response += f"\n\nAction: Please ensure you {action}."
        return response

    # Post-processing on assistant response
    processed_response = enforce_action_items(failure_case['assistant_response'])
    print("\nAssistant Response (After Post-Processing):")
    print(processed_response)


# --- Failure Case 2: Verbosity ---
# We need to make the response more concise.
def analyze_verbosity(failure_case):
    print(f"\nUser Query: {failure_case['user_query']}")
    print(f"Assistant Response (Before): {failure_case['assistant_response']}")
    
    # --- Prompt Template Tweak ---
    # Provide a more concise prompt to the assistant.
    prompt_tweak = """
    User query: "What is the status of my IT support ticket?"
    Assistant response: "Provide a concise update on the ticket status. Avoid unnecessary details and focus on the most relevant information (e.g., current status and estimated resolution time)."
    """
    
    print("\nProposed Prompt Template Tweak (To guide assistant to be more concise):")
    print(prompt_tweak)

    # --- Post-Processing Rule ---
    # Implementing a post-processing rule to limit verbosity.
    def shorten_response(response, max_length=100):
        if len(response) > max_length:
            response = response[:max_length] + "..."
        return response

    # Post-processing on assistant response
    shortened_response = shorten_response(failure_case['assistant_response'])
    print("\nAssistant Response (After Post-Processing - Shortened):")
    print(shortened_response)


# --- Execute the Analysis ---
# Analyzing failure case 1 (missing action items)
analyze_missing_action_items(failure_case_1)

# Analyzing failure case 2 (verbosity)
analyze_verbosity(failure_case_2)



User Query: I can't connect to the company VPN from home, what should I do?
Assistant Response (Before): Please check your network settings and try again.

Proposed Prompt Template Tweak (To guide assistant to provide complete action items):

    User query: "I can't connect to the company VPN from home, what should I do?"
    Assistant response: "Provide a step-by-step troubleshooting guide with clear actions for the user, including things like checking network settings, verifying VPN credentials, restarting the VPN service, and escalation to IT if necessary."
    

Assistant Response (After Post-Processing):
Please check your network settings and try again.

Action: Please ensure you check network.

Action: Please ensure you restart.

Action: Please ensure you contact support.

Action: Please ensure you verify credentials.

User Query: What is the status of my IT support ticket?
Assistant Response (Before): Your support ticket is currently being reviewed by our IT team. They are ana